In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from src.RTScraper import RTScraper

In [14]:
scraper = RTScraper('https://russian.rt.com/rss')
news_df = scraper.get_latest_news()

2023-11-17 21:12:45.865 | INFO     | src.RTScraper:__get_news:63 - Length of news 50
/Users/semionkaminsky/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [15]:
news_df

,title,link,publication_time,text
0,Врач Мясников рассказал о последствиях недоста...,https://russian.rt.com/russia/news/1232745-vra...,2023-11-17 21:09:11+03:00,Врач и телеведущий Александр Мясников рассказа...
1,В Минобороны Узбекистана сообщили о гибели эки...,https://russian.rt.com/ussr/news/1232742-uzbek...,2023-11-17 21:07:56+03:00,Вертолёт Ми-8 потерпел крушение во время трени...
2,Эрдоган напомнил Шольцу о препятствиях в поста...,https://russian.rt.com/world/news/1232743-erdo...,2023-11-17 21:07:28+03:00,Президент Турции Реджеп Тайип Эрдоган в ходе п...
3,"«Плохо, если только они выигрывают»: Путин о в...",https://russian.rt.com/nopolitics/video/123274...,2023-11-17 21:06:20+03:00,"Владимир Путин, выступая на пленарном заседани..."
4,Швед признан лучшим игроком недели в чемпионат...,https://russian.rt.com/sport/news/1232732-shve...,2023-11-17 21:04:14+03:00,Российский баскетболист «Шаньси Лунгс» Алексей...
5,Портнягин назвал «Спартак» одним из самых неэф...,https://russian.rt.com/sport/news/1232737-port...,2023-11-17 21:03:02+03:00,Бывший футболист «Локомотива» Игорь Портнягин ...
6,Большинство россиян поддержали обучение детей ...,https://russian.rt.com/russia/news/1232738-ros...,2023-11-17 21:01:31+03:00,"Согласно новому исследованию, 40% россиян уже ..."
7,"Эрдоган: 13 тысяч детей, женщин и пожилых люде...",https://russian.rt.com/world/news/1232736-erdo...,2023-11-17 20:58:06+03:00,"Более 13 тыс. детей, женщин и пожилых людей бы..."
8,Власти Москвы заявили о росте главных социальн...,https://russian.rt.com/business/news/1232735-m...,2023-11-17 20:54:29+03:00,В январе — сентябре в Москве выросли ключевые ...
9,В центре Москвы неизвестные со стрельбой ограб...,https://russian.rt.com/russia/news/1232734-mos...,2023-11-17 20:54:00+03:00,В центре Москвы около банка несколько человек ...
